In [1]:
# ising modell

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
N = 5
prob = 0.5
lattice = np.random.choice(a=[-1/2, 1/2], size=(N, N), p=[prob, 1-prob])
lattice

array([[-0.5, -0.5, -0.5, -0.5,  0.5],
       [-0.5, -0.5,  0.5,  0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5, -0.5]])

In [4]:
i, j = np.random.choice(a=range(0, N)), np.random.choice(a=range(0, N))
i, j

(2, 0)

In [5]:
lattice = np.random.choice(a=[-1/2, 1/2], size=(N, N), p=[0.0001, 1-0.0001])

def calc_energy(state, J=1., B=1.):
    mu_B = 1.
    
    def pad_with(vector, pad_width, iaxis, kwargs):
        pad_value = kwargs.get('padder', 10)
        vector[:pad_width[0]] = pad_value
        vector[-pad_width[1]:] = pad_value
        return vector
    
    padded_state = np.pad(state, 1, pad_with, padder=0)
    
    spin_neighbor_sum = 0.
    spin_sum = 0.
    for row in range(1, state.shape[0]):
        for col in range(1, state.shape[1]):
            spin_neighbor_sum += padded_state[row, col]*padded_state[row + 1, col]
            spin_neighbor_sum += padded_state[row, col]*padded_state[row - 1, col]
            spin_neighbor_sum += padded_state[row, col]*padded_state[row, col + 1]
            spin_neighbor_sum += padded_state[row, col]*padded_state[row, col - 1]
            spin_sum += padded_state[row, col]
            
    return -J*spin_neighbor_sum - B*spin_sum
    
    
    
calc_energy(lattice)

-22.0

In [6]:
def generate_new_state(state):
    N1, N2 = state.shape
    row, col = np.random.choice(a=range(0, N1)), np.random.choice(a=range(0, N2))
    
    new_state = np.copy(state)
    new_state[row, col] *= -1. # flip the spin of the state

    
    E_current = calc_energy(state)
    E_new = calc_energy(new_state)
    
    if E_new <= E_current:
        return new_state
    
    random_number = np.random.rand()
    relative_prob = np.exp(-(E_new-E_current)/(1.*100.)) # TODO: change to param. T
    
    if relative_prob >= random_number:
        return new_state
    
    return state
    
generate_new_state(lattice)

array([[ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5, -0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5,  0.5]])

In [23]:
import matplotlib.animation as anim
import glob
import moviepy.editor as mpy

def ising(N=80, prob=.45, time=10000):
    
    def calc_energy(state, J=1., B=1.):
        mu_B = 1.
    
        def pad_with(vector, pad_width, iaxis, kwargs):
            pad_value = kwargs.get('padder', 10)
            vector[:pad_width[0]] = pad_value
            vector[-pad_width[1]:] = pad_value
            return vector
    
        padded_state = np.pad(state, 1, pad_with, padder=0)
    
        spin_neighbor_sum = 0.
        spin_sum = 0.
        for row in range(1, state.shape[0]):
            for col in range(1, state.shape[1]):
                spin_neighbor_sum += padded_state[row, col]*padded_state[row + 1, col]
                spin_neighbor_sum += padded_state[row, col]*padded_state[row - 1, col]
                spin_neighbor_sum += padded_state[row, col]*padded_state[row, col + 1]
                spin_neighbor_sum += padded_state[row, col]*padded_state[row, col - 1]
                spin_sum += padded_state[row, col]
            
        return -J*spin_neighbor_sum - B*spin_sum
    
    def generate_new_state(state):
        N1, N2 = state.shape
        row, col = np.random.choice(a=range(0, N1)), np.random.choice(a=range(0, N2))
    
        new_state = np.copy(state)
        new_state[row, col] *= -1. # flip the spin of the state

    
        E_current = calc_energy(state)
        E_new = calc_energy(new_state)
    
        if E_new <= E_current:
            return new_state
    
        random_number = np.random.rand()
        relative_prob = np.exp(-(E_new-E_current)/(1.*3.)) # TODO: change to param. T
    
        if relative_prob >= random_number:
            return new_state
    
        return state
    
    lattice = np.random.choice(a=[-1/2, 1/2], size=(N, N), p=[1, 0])
    
    plt.figure(figsize=(10, 10))
    
    for t in range(0, time):
        plt.imsave(arr=lattice, fname='images/image_%d.png' % t)
        lattice = generate_new_state(lattice)
    plt.imsave(arr=lattice, fname='images/image_%d.png' % time)
    
    gif_name = 'ising'
    fps = 60
    file_list = glob.glob('images/*.png') # Get all the pngs in the current directory
    list.sort(file_list, key=lambda x: int(x.split('_')[1].split('.png')[0])) 
    # Sort the images by #, this may need to be tweaked for your use case
    clip = mpy.ImageSequenceClip(file_list, fps=fps)
    clip.write_gif('{}.gif'.format(gif_name), fps=fps)
    
    print('Gif is saved!')
        
ising()


[MoviePy] Building file ising.gif with imageio


100%|██████████| 10002/10002 [00:26<00:00, 376.56it/s]


Gif is saved!


<Figure size 720x720 with 0 Axes>